In [95]:
import geopandas as gpd
import pandas as pd
import csv
import numpy as np

year = "2017"

# Step 1: Load the GeoJSON files
geojson_file = "./data/NUTS_RG_01M_2024_3035.geojson"
gdf_nuts3 = gpd.read_file(geojson_file)
capitals_geojson_file = "./data/CNTR_RG_20M_2024_3035.geojson"
gdf_capitals = gpd.read_file(capitals_geojson_file)
gdf_capitals = gdf_capitals[["CNTR_ID", "CNTR_NAME", "CAPT"]]

# Step 2: Load the TSV file
tsv_file = "./data/estat_nama_10r_3gdp.tsv"
tsv_file_cleaned = "./output/cleaned_estat_nama_10r_3gdp.tsv"
with open(tsv_file, "r") as file:
    lines = file.readlines()
cleaned_lines = [line.replace(",", "\t") for line in lines]
with open(tsv_file_cleaned, "w") as cleaned_file:
    cleaned_file.writelines(cleaned_lines)
df_gdp = pd.read_csv(tsv_file_cleaned, sep="\t")
df_gdp.rename(columns={"geo\\TIME_PERIOD": "NUTS_ID"}, inplace=True)


In [96]:
def income_country(name):
    try:
        income = df_gdp[gdf_nuts3["NUTS_NAME"] == name][f"{year}"].values[0]
    except:
        income = np.nan
    return income

# income_country("France")
gdf_capitals["country_income"] = gdf_capitals["CNTR_NAME"].apply(income_country)

/tmp/ipykernel_593355/2402488877.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  income = df_gdp[gdf_nuts3["NUTS_NAME"] == name][f"{year}"].values[0]


In [97]:
df_gdp.head(1)

,freq,unit,NUTS_ID,2000,2001,2002,2003,2004,2005,2006,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,A,EUR_HAB,AL,:,:,:,:,:,:,:,...,3300,3500.0,3600,3700,4000,4500,4800,4700,5400 p,:


In [98]:
gdf_nuts3.head(1)

,NUTS_ID,LEVL_CODE,CNTR_CODE,NAME_LATN,NUTS_NAME,MOUNT_TYPE,URBN_TYPE,COAST_TYPE,geometry
0,AT112,3,AT,Nordburgenland,Nordburgenland,None,None,None,"POLYGON ((4847876.719 2800183.532, 4847399.303..."


In [99]:
merged_gdf = gdf_nuts3.merge(df_gdp, left_on="NUTS_ID", right_on="NUTS_ID", how="inner")
merged_gdf.head(2)

,NUTS_ID,LEVL_CODE,CNTR_CODE,NAME_LATN,NUTS_NAME,MOUNT_TYPE,URBN_TYPE,COAST_TYPE,geometry,freq,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,AT112,3,AT,Nordburgenland,Nordburgenland,None,None,None,"POLYGON ((4847876.719 2800183.532, 4847399.303...",A,...,29300,29600.0,30600,31500,32400,33300,34200,33100,34800,:
1,AT112,3,AT,Nordburgenland,Nordburgenland,None,None,None,"POLYGON ((4847876.719 2800183.532, 4847399.303...",A,...,113,111.0,111,112,111,110,109,110,106,:


In [100]:
merged_gdf = gdf_nuts3.merge(df_gdp, left_on="NUTS_ID", right_on="NUTS_ID", how="inner")
merged_gdf = merged_gdf.merge(gdf_capitals, left_on="CNTR_CODE", right_on="CNTR_ID", how="inner")

# Correct column name because there are space " " in name 
for col_year in range(2000,2023):
    merged_gdf[f'{col_year}'] = pd.to_numeric(merged_gdf[f'{col_year} '], errors='coerce')
    merged_gdf.drop(columns=[f'{col_year} '], inplace=True)
    merged_gdf[f'{col_year}'] = merged_gdf[f'{col_year}'].astype(str).str.extract(r'(\d+\.?\d*)')[0].astype(float)

# keep only NUTS_3  (drop NUTS_2 and 1)
# merged_gdf = merged_gdf[merged_gdf["NUTS_ID"].str.len() == 5]
    
# unit : euro_hab
merged_gdf = merged_gdf[merged_gdf['unit'] == "EUR_HAB"]

In [101]:
merged_gdf.head(1)

,NUTS_ID,LEVL_CODE,CNTR_CODE,NAME_LATN,NUTS_NAME,MOUNT_TYPE,URBN_TYPE,COAST_TYPE,geometry,freq,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,AT112,3,AT,Nordburgenland,Nordburgenland,None,None,None,"POLYGON ((4847876.719 2800183.532, 4847399.303...",A,...,29300.0,29600.0,30600.0,31500.0,32400.0,33300.0,34200.0,33100.0,34800.0,NaN


In [103]:
year

2022

In [102]:
# year = "2017"
NUTS_Level = 2

data = merged_gdf[merged_gdf["LEVL_CODE"] == NUTS_Level]

import folium
from folium import Choropleth

# Convert GeoDataFrame to GeoJSON
geojson_data = data.to_crs(epsg=4326).to_json()  # Convert CRS to WGS84 for Folium

# Create a Folium map
m = folium.Map(location=[50, 20], zoom_start=5)  # Adjust to center the map on your region

# Add Choropleth layer
Choropleth(
    geo_data=geojson_data,
    popup=folium.GeoJsonPopup(
        fields=["NUTS_NAME", year],  # Columns to display in popup
        aliases=["Region: ", "GDP: "],  # Popup labels
        localize=True  # Optional: Format large numbers with commas
    ),
    data=data,
    columns=["NUTS_ID", year],  # Columns: key and value
    key_on="feature.properties.NUTS_ID",  # Match GeoJSON property
    fill_color="YlGnBu",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name=f"GDP by NUTS-3 Region for {year}"
).add_to(m)

style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}

hover = folium.features.GeoJson(
    data,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['NUTS_NAME',year],
        aliases=["Region: ", "GDP: "],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
m.add_child(hover)
m.keep_in_front(m)

# Display the map
m.save(f"./output/gdp_{year}_nuts_{NUTS_Level}.html")
# m


KeyError: 2022

In [ ]:
gdf_nuts3[gdf_nuts3["CNTR_CODE"] == "DE"].head(1)

,NUTS_ID,LEVL_CODE,CNTR_CODE,NAME_LATN,NUTS_NAME,MOUNT_TYPE,URBN_TYPE,COAST_TYPE,geometry
158,DE11B,3,DE,Main-Tauber-Kreis,Main-Tauber-Kreis,None,None,None,"POLYGON ((4295787.238 2963164.208, 4295520.607..."


In [ ]:
df_gdp[df_gdp["NUTS_ID"].str.startswith("DE")]

,freq,unit,NUTS_ID,2000,2001,2002,2003,2004,2005,2006,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
224,A,EUR_HAB,DE,25900,26700,26900,27100,27800,28100,29400,...,34900,36100.0,37000,38100,39500,40600,41800,40900 p,43500 p,46300 p
225,A,EUR_HAB,DE1,29800,31000,31100,31400,31700,31900,34000,...,40100,41500.0,42900,43500,45300,46800 e,47400 e,45900 e,48700 e,51300 e
226,A,EUR_HAB,DE11,33600,35400,35200,35800,35800,35600,38300,...,45600,47000.0,49200,49400,51800,53700 e,53300 e,51400 e,54500 e,57300 e
227,A,EUR_HAB,DE111,61700,67100,69200,71600,70900,69000,74200,...,78200,79800.0,84900,83700,87100,91800 e,88100 e,83800 e,88300 e,:
228,A,EUR_HAB,DE112,38600,42200,40300,40900,39500,35500,40700,...,56100,57600.0,63500,63400,66200,66800 e,62500 e,60400 e,67200 e,:
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11370,A,PPS_HAB_EU27_2020,DEG0K,70,70,75,75,75,71,72,...,86,89.0,84,84,88,90 e,88 e,93 e,94 e,:
11371,A,PPS_HAB_EU27_2020,DEG0L,64,64,62,65,65,60,62,...,67,69.0,67,70,72,73 e,70 e,69 e,67 e,:
11372,A,PPS_HAB_EU27_2020,DEG0M,56,56,55,59,59,57,58,...,65,70.0,67,68,68,67 e,66 e,68 e,66 e,:
11373,A,PPS_HAB_EU27_2020,DEG0N,140,134,158,164,153,136,139,...,114,139.0,137,133,130,120 e,113 e,111 e,108 e,:


In [ ]:
merged_gdf.keys()

Index(['NUTS_ID', 'LEVL_CODE', 'CNTR_CODE', 'NAME_LATN', 'NUTS_NAME',
       'MOUNT_TYPE', 'URBN_TYPE', 'COAST_TYPE', 'geometry', 'freq', 'unit',
       '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017',
       '2018', '2019', '2020', '2021', '2022'],
      dtype='object')

In [ ]:
merged_gdf.to_csv(f"./output/gdp_{year}.csv")